<a href="https://colab.research.google.com/github/jaeofbum/-ViT-/blob/main/Resnet50_tuning_yjb_(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from tensorflow.keras.utils import plot_model

In [ ]:
#ResNet50 불러오기
base_model = keras.applications.resnet.ResNet50(
        weights= 'imagenet',
        include_top=False)

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
#ResNet50 동결시키기
base_model.trainable = False

In [ ]:
#ResNet50 모델에 input size 설정 및 계층 추가
inputs = keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(2)(x)
model = keras.Model(inputs, outputs)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#train,valid,test 데이터셋 불러오기
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train_gen = ImageDataGenerator(horizontal_flip=True, rescale=1/255,rotation_range=10,width_shift_range=0.1, height_shift_range=0.1,shear_range=0.1,zoom_range=0.1)
train_gen = ImageDataGenerator(rescale=1/255.)
# Test ImageDataGenerator
test_gen = ImageDataGenerator(rescale=1/255.)
valid_gen = ImageDataGenerator(rescale=1/255.)

train_flow_gen = train_gen.flow_from_directory(directory='/content/drive/MyDrive/Colab Notebooks/classification/train',
                                              target_size=(224, 224),  
                                              class_mode='categorical',
                                              batch_size=16,
                                              shuffle=True)
valid_flow_gen = test_gen.flow_from_directory(directory='/content/drive/MyDrive/Colab Notebooks/classification/val',
                                            target_size=(224, 224), 
                                            class_mode='categorical',
                                            batch_size=16,
                                            shuffle=False)
test_flow_gen = test_gen.flow_from_directory(directory='/content/drive/MyDrive/Colab Notebooks/classification/test',
                                            target_size=(224, 224), 
                                            class_mode='categorical',
                                            batch_size=16,
                                            shuffle=False)

Found 394 images belonging to 2 classes.
Found 132 images belonging to 2 classes.
Found 132 images belonging to 2 classes.
Found 394 images belonging to 2 classes.
Found 132 images belonging to 2 classes.
Found 132 images belonging to 2 classes.


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
 Layer (type)               

In [ ]:
model.evaluate(test_flow_gen)

9/9 [==============================] - 3s 289ms/step - loss: 0.3213 - accuracy: 0.9091


[0.32133713364601135, 0.9090909361839294]

In [ ]:
#resnet50을 제외한 부분을 미리 학습시키기
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
    loss=keras.losses.CategoricalCrossentropy(
        from_logits=True))
model.fit(train_flow_gen, epochs=20 ,validation_data=valid_flow_gen)  

Epoch 1/20
25/25 [==============================] - 235s 9s/step - loss: 0.6863 - accuracy: 0.5660 - val_loss: 0.6775 - val_accuracy: 0.6515
Epoch 2/20
25/25 [==============================] - 11s 423ms/step - loss: 0.6789 - accuracy: 0.5736 - val_loss: 0.6657 - val_accuracy: 0.6667
Epoch 3/20
25/25 [==============================] - 11s 426ms/step - loss: 0.6668 - accuracy: 0.6142 - val_loss: 0.6533 - val_accuracy: 0.7273
Epoch 4/20
25/25 [==============================] - 11s 435ms/step - loss: 0.6638 - accuracy: 0.5660 - val_loss: 0.6455 - val_accuracy: 0.5909
Epoch 5/20
25/25 [==============================] - 11s 419ms/step - loss: 0.6532 - accuracy: 0.6041 - val_loss: 0.6309 - val_accuracy: 0.7348
Epoch 6/20
25/25 [==============================] - 11s 434ms/step - loss: 0.6381 - accuracy: 0.6447 - val_loss: 0.6209 - val_accuracy: 0.7955
Epoch 7/20
25/25 [==============================] - 11s 430ms/step - loss: 0.6360 - accuracy: 0.6142 - val_loss: 0.6231 - val_accuracy: 0.5909
E

Epoch 1/20
25/25 [==============================] - 14s 478ms/step - loss: 0.7591 - accuracy: 0.4721 - val_loss: 0.6962 - val_accuracy: 0.5000
Epoch 2/20
25/25 [==============================] - 11s 425ms/step - loss: 0.6945 - accuracy: 0.5254 - val_loss: 0.6743 - val_accuracy: 0.6667
Epoch 3/20
25/25 [==============================] - 10s 428ms/step - loss: 0.6859 - accuracy: 0.5330 - val_loss: 0.6749 - val_accuracy: 0.5076
Epoch 4/20
25/25 [==============================] - 11s 411ms/step - loss: 0.6783 - accuracy: 0.5635 - val_loss: 0.6570 - val_accuracy: 0.6136
Epoch 5/20
25/25 [==============================] - 11s 428ms/step - loss: 0.6716 - accuracy: 0.5711 - val_loss: 0.6462 - val_accuracy: 0.6515
Epoch 6/20
25/25 [==============================] - 11s 427ms/step - loss: 0.6520 - accuracy: 0.6320 - val_loss: 0.6358 - val_accuracy: 0.7273
Epoch 7/20
25/25 [==============================] - 11s 428ms/step - loss: 0.6491 - accuracy: 0.6193 - val_loss: 0.6260 - val_accuracy: 0.8409

In [ ]:
#resnet50 동결풀기
model.trainable = True
#early stopping 넣기
callback=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
    mode="auto",
    restore_best_weights=True,
)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
    metrics=["accuracy"],
    loss=keras.losses.CategoricalCrossentropy(from_logits=True))
model.fit(train_flow_gen, epochs=10 ,callbacks=[callback],validation_data=valid_flow_gen)

Epoch 1/10
25/25 [==============================] - 17s 489ms/step - loss: 0.1454 - accuracy: 0.9391 - val_loss: 0.1531 - val_accuracy: 0.9318
Epoch 2/10
25/25 [==============================] - 11s 443ms/step - loss: 0.1449 - accuracy: 0.9391 - val_loss: 0.2792 - val_accuracy: 0.8864
Epoch 3/10
25/25 [==============================] - 11s 406ms/step - loss: 0.0562 - accuracy: 0.9898 - val_loss: 0.2805 - val_accuracy: 0.8939


In [ ]:
#모델 평가하기
model.evaluate(test_flow_gen)

9/9 [==============================] - 2s 271ms/step - loss: 0.2968 - accuracy: 0.8939


[0.2967815399169922, 0.8939393758773804]

In [ ]:
#모델 평가하기
model.evaluate(test_flow_gen)

9/9 [==============================] - 2s 268ms/step - loss: 0.2376 - accuracy: 0.9167


[0.2375577688217163, 0.9166666865348816]

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
#모델 저장하기
filepath = '/content/drive/MyDrive/Colab Notebooks/resnet50 yjb test.h5'
tf.keras.models.save_model(
    model, filepath, overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
)